In [35]:
import numpy as np
from time import sleep, time

import serial
from datetime import datetime

import os
import sys
import preprocessing as pp
from IPython.display import clear_output

In [88]:
print("Script started")

num_sensors = 96
run_duration = 600

try:
    # Serial setup
    ser = serial.Serial('COM3', 230400, timeout=1.0)
    print("Serial is ready")
    ser.flushInput()
except:
    pass

all_data = []

Script started
Serial is ready


In [89]:
obj = '5'
file_number = '1'


txt_file = f'data_obj{obj}_{file_number}.txt'
base_directory = "train_stiffness/"

path_txt = os.path.join(base_directory, txt_file)

if os.path.exists(path_txt):
    print(f"Error: file already exists data_obj{obj}_.txt/h5.")
    if ser.is_open:
        ser.close()
        print("Serial port closed successfully after data integrity check.")
    sys.exit()

In [90]:
def read_serial_binary():
    """Read 96 binary values (2 bytes each) + checksum between < and > markers."""
    expected_bytes = num_sensors * 2 + 2  # 96 values × 2 bytes + 2 bytes for checksum
    buffer = b""
    start_time = time()
    timeout = 0.5 
    # Wait for start marker
    while True:
        if (time() - start_time) > timeout:
            print("Warning: Timeout waiting for start marker")
            return None
        char = ser.read(1)
        if char == b'<':
            break
        elif char == b'':
            continue
    
    # Read data and checksum
    data = ser.read(expected_bytes)
    buffer += data
    
    # Wait for end marker
    char = ser.read(1)
    if char != b'>':
        print("Warning: End marker not found")
        return None
    
    if len(data) != expected_bytes:
        print(f"Warning: Expected {expected_bytes} bytes, got {len(data)}")
        return None
    
    # Extract values and checksum
    values = np.frombuffer(data[:num_sensors * 2], dtype=np.uint16)  # 96 values
    received_checksum = np.frombuffer(data[num_sensors * 2:], dtype=np.uint16)[0]
    
    # Verify checksum
    computed_checksum = np.sum(values, dtype=np.uint16)
    if computed_checksum != received_checksum:
        print(f"Warning: Checksum mismatch - computed: {computed_checksum}, received: {received_checksum}")
        return None
    
    return values

def main_loop():
    global all_data, obj, txt_file, iteration_count

    ser.flushInput()
    with open(path_txt, "w") as txtfile:
        labels = []
        start_time = time()
        iteration_count = 0
        while True:
            # Check timer at the start of each iteration
            elapsed_time = time() - start_time
            if elapsed_time >= run_duration:
                print(f"Timer expired after {elapsed_time:.2f} seconds. Stopping data collection.")
                break
            ser.flushInput()
            count = 0
            sample_data = np.zeros((30, 96))
            while count < 30:
                ser.write(b'R')
                ser.flush()
                values = read_serial_binary()
                if values is None or len(values) != num_sensors:
                    print(f"Warning: Expected {num_sensors} values, got {len(values) if values is not None else 0}")
                    continue
                if all(value <= 1000 for value in values):
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')
                    txtfile.write(f"{values},{timestamp}\n")
                    iteration_count += 1
                else:
                    print("Warning: One or more sensor values exceeded the maximum allowed value of 1000.")
                    continue
                numeric_arrays = np.where(values > 15, values, 0)
                sample_data[count] = numeric_arrays
                count += 1
            thresh = 0.0023
            thresh = 0.00
            resistances = pp.transform_to_resistances(sample_data)
            smooth_resistances = pp.robust_smoothing(resistances)
            power = pp.compute_power(smooth_resistances)
            if power > thresh:
                all_data.append(smooth_resistances[np.newaxis, :])
                clear_output()
                print(len(all_data))
            else:
                clear_output()
                print("Пусто")
                continue

    print(np.concatenate(all_data, axis=0).shape)
    np.save(f'train_stiffness/obj{obj}_{file_number}.txt', np.concatenate(all_data, axis=0))

In [91]:
if __name__ == '__main__':
    print("Start")
    main_loop() 
    if ser.is_open:
        ser.close()
        print("Serial port closed successfully after data integrity check.")

679
Timer expired after 600.18 seconds. Stopping data collection.
(679, 30, 96)
Serial port closed successfully after data integrity check.


In [44]:
a = np.load('train/obj2_1.txt.npy')
b = np.load('train/obj0_1.txt.npy')
c = np.load('train/obj1_1.txt.npy')

In [45]:
print(a.shape, b.shape, c.shape)

(512, 30, 96) (739, 30, 96) (594, 30, 96)
